# Future Sales Prediction - Kaggle Competition

https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales/

#### File descriptions
- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.

- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.

- sample_submission.csv - a sample submission file in the correct format.

- items.csv - supplemental information about the items/products.

- item_categories.csv  - supplemental information about the items categories.

- shops.csv- supplemental information about the shops.

#### Data fields

- ID - an Id that represents a (Shop, Item) tuple within the test set

- shop_id - unique identifier of a shop

- item_id - unique identifier of a product

- item_category_id - unique identifier of item category

- item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

- item_price - current price of an item

- date - date in format dd/mm/yyyy

- date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

- item_name - name of item

- shop_name - name of shop

- item_category_name - name of item category

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
sales_train = pd.read_csv("data/sales_train.csv")
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [89]:
test = pd.read_csv("data/test.csv")
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [90]:
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [91]:
items = pd.read_csv("data/items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [92]:
item_categories = pd.read_csv("data/item_categories.csv")
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [93]:
shops = pd.read_csv("data/shops.csv")
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


### Initial data checks and data manipulation

In [94]:
sales_train.shape

(2935849, 6)

In [95]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


Note that the ```date``` column is not in the proper datetime format.

In [96]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format = '%d.%m.%Y')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [97]:
# Split into month, day, and year
sales_train['year'] = sales_train['date'].dt.year
sales_train['month'] = sales_train['date'].dt.month
sales_train['day'] = sales_train['date'].dt.day
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,2
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,3
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,5
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,6
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,15


In [105]:
# Keep the month-year pair to make date_block_num column more intelligible
sales_train['month_year'] = sales_train['date'].dt.strftime('%Y-%m-%d').str[:7]
# dt.strftime(...) to convert the datetime format of 'date' into string format
# str[:7] to extract the month and year in the date string

# Uncomment this command to convert month_year to datetime format
# sales_train['month_year'] = pd.to_datetime(sales_train['month_year'], format = '%Y-%m')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,month_year
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,2,2013-01
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,3,2013-01
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,5,2013-01
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,6,2013-01
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,15,2013-01


In [99]:
sales_train.isna().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
year              0
month             0
day               0
month_year        0
dtype: int64

In [100]:
sales_train.nunique()

date               1034
date_block_num       34
shop_id              60
item_id           21807
item_price        19993
item_cnt_day        198
year                  3
month                12
day                  31
month_year           34
dtype: int64

In [101]:
sales_train.describe()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,month_year
count,2935849,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2935849
mean,2014-04-03 05:44:34.970681344,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,2.013777e+03,6.247717e+00,1.585267e+01,2014-03-19 09:16:44.528911104
min,2013-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,2.013000e+03,1.000000e+00,1.000000e+00,2013-01-01 00:00:00
25%,2013-08-01 00:00:00,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.013000e+03,3.000000e+00,8.000000e+00,2013-08-01 00:00:00
50%,2014-03-04 00:00:00,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,2.014000e+03,6.000000e+00,1.600000e+01,2014-03-01 00:00:00
75%,2014-12-05 00:00:00,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,2.014000e+03,9.000000e+00,2.400000e+01,2014-12-01 00:00:00
max,2015-10-31 00:00:00,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,2.015000e+03,1.200000e+01,3.100000e+01,2015-10-01 00:00:00
std,NaN,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,7.684790e-01,3.536219e+00,8.923483e+00,NaN


In [103]:
# Create a "master table" for EDA in training data
sales_full = sales_train.merge(items, how = 'inner', on = 'item_id')\
                              .merge(item_categories, how = 'inner', on = 'item_category_id')\
                              .merge(shops, how = 'inner', on = 'shop_id')

sales_full.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,month_year,item_name,item_category_id,item_category_name,shop_name
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,2,2013-01-01,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,3,2013-01-01,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,5,2013-01-01,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,6,2013-01-01,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,15,2013-01-01,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [110]:
sales_train_by_month = sales_train[['month_year', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']].groupby(['month_year', 'shop_id', 'item_id', 'item_price']).sum()
sales_train_by_month

item_cnt_day
month_year shop_id item_id item_price              
2013-01    0       32      221.0                6.0
                   33      347.0                3.0
                   35      247.0                1.0
                   43      221.0                1.0
                   51      127.0                1.0
...                                             ...
2015-10    59      22087   119.0                6.0
                   22088   119.0                2.0
                   22091   179.0                1.0
                   22100   629.0                1.0
                   22102   1250.0               1.0

[1739022 rows x 1 columns]